In [52]:
import os
import datetime
import smtplib
from email.message import EmailMessage
import locale
import calendar
import pandas as pd
import pyodbc
import xlwings as xw
from openpyxl.utils import get_column_letter


**Download Exchange Rate from Yahoo Finance**

In [53]:
import yfinance as yf

# Definir el ticker para el tipo de cambio USD/COP
ticker = "COP=X"

# Descargar los datos históricos
# data = yf.download(ticker, period="1d", interval="1d")
data = yf.download(ticker, period="max", interval="1wk")

# Restablecer el índice para que las fechas sean una columna
data.reset_index(inplace=True)


[*********************100%%**********************]  1 of 1 completed


**Visualize Data**

In [54]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2003-01-01,2852.629883,2879.399902,2835.000000,2838.500000,2838.500000,0
1,2003-04-16,2919.899902,2925.000000,2917.000000,2919.399902,2919.399902,0
2,2003-11-26,2777.500000,2777.500000,2765.000000,2765.000000,2765.000000,0
3,2003-12-03,2751.800049,2765.600098,2750.600098,2755.399902,2755.399902,0
4,2003-12-10,2763.699951,2763.699951,2739.000000,2739.000000,2739.000000,0
...,...,...,...,...,...,...,...
1076,2024-06-26,4088.729980,4178.229980,4088.386475,4132.250000,4132.250000,0
1077,2024-07-03,4117.950195,4123.410156,4023.449951,4064.750000,4064.750000,0
1078,2024-07-10,4046.399902,4046.399902,3925.750000,3935.199951,3935.199951,0
1079,2024-07-17,3975.360107,4046.719971,3960.810059,3992.250000,3992.250000,0


**Open the Excel Workbook template**

In [55]:
# ruta_plantilla_excel= r"C:\Users\Juan Esteban\Desktop\Proyectos Python\02. Automatizaciones\00.(Template) Precio del dolar.xlsx"
# plantilla_excel= xw.Book(ruta_plantilla_excel)
# resumen= plantilla_excel.sheets['Hoja1']

Get the current directory where the code is being executed

In [56]:
import os
import xlwings as xw

current_directory = os.getcwd()

Define the path for the new file:

In [57]:
# Define the path for the new Excel file
new_excel_path = os.path.join(current_directory, 'NewFile.xlsx')


Create the new Excel file:

In [58]:
# Create a new Excel workbook
plantilla_excel = xw.Book()

resumen= plantilla_excel.sheets['Hoja1']

**Insert the DataFrame Information into the Excel File**


In [59]:
# Assuming 'data' is a DataFrame, get the list of column names
columnas = data.columns.tolist()

# Write the column names to cell B2 in the worksheet
resumen.range('B2').value = columnas

# Write the DataFrame values starting from cell B3 in the worksheet
resumen.range('B3').value = data.values

**Define the range for the header and apply the color:**

In [60]:
# Define the range for the header (first row of the table)
header_range = resumen.range('B2:H2')

# Convert hex color to RGB
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# Apply color to the header range
header_color_rgb = hex_to_rgb('#305496')
header_range.api.Interior.Color = (header_color_rgb[2] << 16) | (header_color_rgb[1] << 8) | header_color_rgb[0]

# Apply font color to the header range
header_range.api.Font.Color = 0xFFFFFF  # White color in hex

**Add Border to the Table**

In [61]:
rango = resumen.range(f'B2:H{len(data)+2}')#rango total tabla
rango.api.Borders.LineStyle = xw.constants.LineStyle.xlContinuous#ponerle borde a todo

**Apply the currency format to numeric columns.**

In [62]:
# Define the range for columns C to H, starting from row 2 to the last row with data
data_range = resumen.range(f'C2:H{len(data)+1}')

# Apply currency format without decimal places to the range
data_range.number_format = '$#,##0'

**Create a Graph**

In [63]:
# Define the data range for the chart
rango_datos = resumen.range(f'B2:C{data.shape[0] + 2}')

# Create a chart in a specific location, for example, in cell G2
chart = resumen.charts.add(left=resumen.range('J2').left, top=resumen.range('J2').top, width=600, height=400)

# Set the data source for the chart
chart.set_source_data(rango_datos)

# Set the chart type, for example, line
chart.chart_type = 'line'

# Add a title to the chart
chart.api[1].HasTitle = True
chart.api[1].ChartTitle.Text = 'Precio del Dólar (USD/COP)'


**Save the File with a New Name**

In [64]:
#fechas
fecha_hoy = datetime.datetime.today()#datetime.date(2023, 9, 17)
fecha_comienzo_mes = fecha_hoy.replace(day = 1)
fecha_ultimo_comite = fecha_comienzo_mes - datetime.timedelta(days = 1)

fecha_ultimo_comite
## Configurar el idioma en español
locale.setlocale(locale.LC_TIME, 'es_ES.utf8')
nombre_mes_actual = calendar.month_name[fecha_ultimo_comite.month]

nombre_mes_actual

'junio'

Ponerle nombre al archivo y cerrarlo

In [65]:

plantilla_excel.save(f"1. Historico de tasas de cambio {nombre_mes_actual.capitalize()} {fecha_ultimo_comite.year}.xlsx")

# Cerrar el archivo Excel
plantilla_excel.close()